In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [ ]:
fn = "../subworkflows/pypsa-earth-sec/data/industrial_database.csv"

In [ ]:
idb = pd.read_csv(fn, index_col=0)

In [ ]:
idb.head()

In [ ]:
# Filter for country MA
idb = idb[idb.index == "MA"]

In [ ]:
px.scatter_mapbox(
    idb,
    lat="y",
    lon="x",
    mapbox_style="carto-positron",
    color="technology",
    #size="capacity",
    # size
    #hover_name="name",
    zoom=4,
    height=600,
)

In [ ]:
# Plot with plotly express

fig = px.scatter_mapbox(idb, lat="y", lon="x", hover_name="location", hover_data=["technology", "capacity"],
                        color="technology", zoom=1, height=300)
fig.update_layout(mapbox_style="open-street-map")
fig.update_layout(margin={"r":0,"t":0,"l":0,"b":0})

# increase figsize
fig.update_layout(
    autosize=False,
    width=800,
    height=800,
    margin=dict(
        l=50,
        r=50,
        b=100,
        t=100,
        pad=4
    ),
    paper_bgcolor="LightSteelBlue",
)

fig.show()